# Bini Fast Matrix Multiplication O(2.7799)

In [1]:
import numpy as np

In [2]:
from numpy import linalg as la

In [3]:
np.set_printoptions(precision=2)

In [4]:
#without dynamic peeling:
# row of A need to be divisible by 3 (power of is preferable)
# columns of A need do be divisble by 2

# rows and cols of B need to be divisible by two

In [102]:
def calculate_e(steps):
    e = (2**-52)**(1/(1+steps))
    return e

In [92]:
def bini(A, B, steps, e=1e-8):
    
    #Check Dimensions
    (m, n) = A.shape
    #rn assuming that m is bigger than n, nn and p
    (nn, p) = B.shape
    if n != nn: raise ValueError("incompatible dimensions")
    
    #pre-allocate output matrix
    C = np.zeros((m,p))
    
    """
    This is the notation I use from Bini's 1980 paper

    |A1, A4|  |B1, B2|  =  |C1, C2|
    |A2, A5|  |B3, B4|     |C3, C4|
    |A3, A6|               |C5, C6|
    """
    
    #Base case
    if steps == 0 or m == 1 or n == 1 or p == 1:
        C = np.dot(A,B)
        return C
    
    #Static peeling
    if (3**steps > m) or (2**steps > n) or (2**steps > p):
        raise ValueError("Too many steps/ too small matricies for static peeling")
    
    if (m % 3**steps) != 0:
        extra_rows = m % 3**steps
        
        C[:m-extra_rows, :] = bini(A[:m-extra_rows, :], B, steps, e)
        C[m-extra_rows:, :] = A[m-extra_rows:, :]@B
        return C
    if (n % 2**steps) != 0:
        extra_cols = n % (2**steps)
        
        C = bini(A[:, :n-extra_cols], B[:n-extra_cols,:], steps, e)
        C = C + A[:, n-extra_cols:]@B[n-extra_cols:, :]
        return C
    if (p % 2**steps) != 0:
        multiP = p//(2**steps) #multipler to find how large to make the bini matrix
        extra_cols = p % (2**steps)
        
        C[:, :p-extra_cols] = bini(A, B[:, :p-extra_cols], steps, e)
        C[:, p-extra_cols:] = A@B[:, p-extra_cols:]
        return C
    
    
    """
    Dynamic peeling causes issues because the ideal epsilon value is determined by 
    the shape of the matrix and in dynamic peeling, the shape of the matrix
    is changed every recursive step which results in dimensions with a different 
    ideal epsilon value
    
    #Dynamic peeling
    if m % 3 == 1:
        C[:m-1, :] = bini(A[:m-1,:],B, steps, e)
        C[m-1,:] = A[m-1,:]@B
        return C
    if m % 3 == 2:
        C[:m-2, :] = bini(A[:m-2,:],B, steps, e)
        C[m-2:,:] = A[m-2:,:]@B
        return C
    if n % 2 == 1:
        C = bini(A[:, :n-1], B[:n-1,:], steps, e)
        C = C + np.outer(A[:,n-1],B[n-1,:])
        return C
    if p % 2 == 1:
        C[:, :p-1] = bini(A, B[:,:p-1], steps, e)
        C[:,p-1] = A@B[:,p-1]
        return C
    """
 

    # split up the matricies once rows of A are divisible by 3
    # and cols of A and rows and cols of are divisible by 2
    m2 = int(m/3) #first third of the rows of A
    m3 = m2*2     #second third of the rows of A
    n2 = int(n/2) #half of the cols of A
    p2 = int(p/2) #half of the cols of B
    #nn2 = int(nn/2) # half of the rows of B
    
    A1 = A[:m2, :n2]
    A2 = A[m2:m3, :n2]
    A3 = A[m3:, :n2]
    A4 = A[:m2, n2:]
    A5 = A[m2:m3, n2:]
    A6 = A[m3:, n2:]
    
    B1 = B[:n2, :p2]
    B2 = B[:n2, p2:]
    B3 = B[n2:, :p2]
    B4 = B[n2:, p2:]
    
    #bini(A, B, steps, e=0.1)
    # conquer
    M1 = bini(A1 + A5, e*B1 + B4, steps-1, e) 
    M2 = bini(A5, -B3-B4, steps-1, e)
    M3 = bini(A1, B4, steps-1, e)
    M4 = bini(e*A4+A5, -e*B1 + B3, steps-1, e)
    M5 = bini(A1 + e*A4, e*B2 + B4, steps-1, e)
    M6 = bini(A2 + A6, B1 + e*B4, steps-1, e)
    M7 = bini(A2, -B1 - B2, steps-1, e) #
    M8 = bini(A6, B1, steps-1, e)
    M9 = bini(A2 + e*A3, B2 - e*B4, steps-1, e)
    M10 = bini(e*A3 + A6, B1 + e*B3, steps-1, e)
    
    # put C together
    C[:m2, :p2] = (1/e)*(M1+M2-M3+M4) #C1
    C[:m2, p2:] = (1/e)*(-M3+M5)      #C2
    C[m2:m3, :p2] = M4+M6-M10         #C3 error from bini paper -M10 from +M10
    C[m2:m3, p2:] = M1-M5+M9          #C4 error from bini paper -M5 from +M5
    C[m3:, :p2] = (1/e)*(-M8+M10)     #C5
    C[m3:, p2:] = (1/e)*(M6+M7-M8+M9) #C6
    
    return C

In [93]:
A = np.random.rand(243, 256)
B = np.random.rand(256, 256)

In [96]:
steps = 1

In [103]:
calculate_e(steps)

1.4901161193847656e-08

In [104]:
C = bini(A,B, steps, calculate_e(steps))

In [105]:
la.norm(C-A@B, 'fro')/la.norm(C)

2.0653832898209107e-08

In [85]:
for i in range(20):
    a = -1*(i+1)
    C = bini(A,B, 3, e=10**a)
    error = la.norm(C-A@B, 'fro')/la.norm(C) 
    print("e:", 10**a, "Error:", error)

e: 0.1 Error: 0.07280892626911609
e: 0.01 Error: 0.007155594969398102
e: 0.001 Error: 0.0007154526021214912
e: 0.0001 Error: 0.0001557402385775404
e: 1e-05 Error: 0.13683466979819325
e: 1e-06 Error: 0.9999671136808329
e: 1e-07 Error: 0.9999999985713249
e: 1e-08 Error: 0.9999999999856622
e: 1e-09 Error: 0.9999999999999855
e: 1e-10 Error: 0.9999999999999998
e: 1e-11 Error: 1.0
e: 1e-12 Error: 1.0
e: 1e-13 Error: 1.0
e: 1e-14 Error: 1.0
e: 1e-15 Error: 1.0
e: 1e-16 Error: 1.0
e: 1e-17 Error: 1.0
e: 1e-18 Error: 1.0
e: 1e-19 Error: 1.0
e: 1e-20 Error: 1.0


1.0

In [46]:
 3**-3 == 1.0/3**3

True